<a href="https://colab.research.google.com/github/Aleena24/Large-Language-Model/blob/main/lab1_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

In [2]:
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Split the dataset into train and validation sets
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
def preprocess_data(examples):
    questions = examples['question']
    contexts = examples['context']
    inputs = tokenizer(
        questions,
        contexts,
        max_length=512,  # Adjust this if needed
        truncation="longest_first",
        padding="max_length",
        return_tensors="pt"
    )

    start_positions = []
    end_positions = []

    for i in range(len(examples['answers'])):
        start_positions.append(examples['answers'][i]['answer_start'][0])
        end_positions.append(examples['answers'][i]['answer_start'][0] + len(examples['answers'][i]['text'][0]))

    inputs.update({
        'start_positions': torch.tensor(start_positions),
        'end_positions': torch.tensor(end_positions)
    })

    return inputs

# Preprocess the train and validation datasets
train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
valid_dataset = valid_dataset.map(preprocess_data, batched=True, remove_columns=valid_dataset.column_names)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# Convert to torch dataset
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'start_positions', 'end_positions'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

# Optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

# Training loop
model.train()
for epoch in range(3):  # Let's train for 3 epochs
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} finished")


In [ ]:
model.eval()
with torch.no_grad():
    for batch in valid_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

        # Here you can add your own logic to calculate metrics
        # e.g., accuracy, F1 score, etc.


In [ ]:
# Define a context and a question
context = "Hugging Face is a technology company based in New York and Paris."
question = "Where is Hugging Face based?"

# Tokenize the inputs
inputs = tokenizer.encode_plus(question, context, return_tensors="pt").to(device)

# Get the model's predictions
with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

# Convert the logits to predicted start and end positions
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits)

# Decode the answer from the context
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
answer = tokenizer.convert_tokens_to_string(tokens[start_idx:end_idx+1])

print(f"Question: {question}")
print(f"Answer: {answer}")
